# **Avaliação 03 - ENGA74 2023.1**
**Aluno:** André Paiva Conrado Rodrigues

## **Importação de dependências para os códigos**

In [ ]:
import numpy as np
from numpy.random import rand
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## **Funções de ***benchmark*****

Esfera:

In [ ]:
#Função Esfera
def esfera(x_array):
    return np.sum(np.power(x_array, 2))

#Derivadas da Esfera
def deriv_esfera(x_array):
    return 2 * x_array

Ackley:

In [ ]:
#Função Ackley
def ackley(x_array):
    exp1 = np.exp(-0.2 * np.sqrt(np.sum(np.power(x_array, 2))/x_array.shape[0]))
    exp2 = np.exp(np.sum(np.cos(2 * np.pi * x_array))/x_array.shape[0])
    return (-20 * exp1) - exp2 + 20 + np.e

#Derivadas de Ackley
def deriv_ackley(x_array):
    aux = x_array/np.sqrt(np.sum(np.power(x_array, 2)))
    exp1 = np.exp(-0.2 * np.sqrt(np.sum(np.power(x_array, 2))/x_array.shape[0]))
    exp2 = np.exp(np.sum(np.cos(2 * np.pi * x_array))/x_array.shape[0])
    coef1 = 2.828
    coef2 = np.pi
    if(x_array.shape[0] == 3):
        coef1 = 5.6562/np.sqrt(3)
        coef2 = 2 * np.pi / 3
    return (coef1 * aux * exp1) + (coef2 * exp2 * np.sin(2 * np.pi * x_array))
